### Mimic Pivot Table for Tracker Data

In [10]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('All_results_with_res_groups.csv', sep=',', index_col=None)

In [20]:
df.head()

,Clip,Res,Res Group,Tracker,IOU_Thresh,Dist_Thresh,SOTP,SOTA,Recall,Precision,...,FPS,fp_count,fp_thresh_count,tp_count,fn_count,tn_count,N_objects,N_frames,N_match,Tracked_Ratio
0,Bike03,1504,High,csrt,0.4,0.1,0.0300,1.0000,1.0000,0.7865,...,18.70,19,0,70,0,0,70,90,70,1.000
1,Slalom01,1504,High,csrt,0.4,0.1,0.1390,0.4242,0.4242,0.4242,...,22.88,0,57,42,0,0,99,100,99,1.000
2,Ski03,1920,High,csrt,0.4,0.1,0.0130,1.0000,1.0000,1.0000,...,15.61,0,0,1113,0,0,1113,1114,1113,1.000
3,Bike09,1920,High,csrt,0.4,0.1,0.0177,0.4218,0.4218,0.9286,...,16.67,11,0,143,196,86,339,437,143,0.422
4,Ski04,1920,High,csrt,0.4,0.1,0.0116,1.0000,1.0000,1.0000,...,16.22,0,0,253,0,0,253,254,253,1.000


In [27]:
pd.pivot_table(df, index=['Tracker'], columns=['Res Group'], aggfunc=np.mean, values=['SOTP'], margins=True)

SOTP                              
Res Group       High       Low       Mid       All
Tracker                                           
csrt        0.016517  0.012767  0.041028  0.023437
goturn      0.234024  0.258129  0.306341  0.266165
kcf         0.010072  0.008200  0.009472  0.009248
medianflow  0.136200  0.206894  0.167217  0.170104
mil         0.112506  0.089639  0.110472  0.104206
mosse       0.025418  0.028060  0.019771  0.023836
tld         0.209228  0.242356  0.183872  0.211819
All         0.105902  0.126036  0.119040  0.116819